# 언어판별

In [52]:
# 모듈 불러오기
import os

## 데이터 명세
* Given
  * 영어, 프랑스어, 인도네시아어, 타갈로그어로 된 글이 주어짐
  * 국가명을 알 수 있는 글로 된 학습 데이터 20개 / 테스트 데이터 8개
  * 파일 이름은 "국가명(두 단어)"-"데이터번호".txt 형태
* Goal
  * 새로운 글이 주어지면 어느 국가의 언어로 쓰인 글인지 판별
* 머신러닝 분류
  * 관측한 데이터(글)가 어느 집합(영어, 프랑스어, 인도네시아어, 타갈로그어)에 속하는지 알고 있음 -> 지도학습
  * 어느 범주에 속하는지 **분류**
  * 즉, 지도학습 -> 분류 문제
  * 풀이 방법: 의사결정 나무, 서포트벡터머신

## 파일 리스트 만들기

* 파일을 읽어올 경로를 정해줌
* os.listdir(path)
  * path 위치에 있는 디렉토리의 요소를 모두 반환
  * 요소의 순서는 정해지지 않음
  * 단, '.'과 '..'는 제외

In [19]:
train_path = '../data/lang/train'
test_path = '../data/lang/test'

In [16]:
# 학습 데이터 위치의 파일 리스트
os.listdir(train_path)

['.DS_Store',
 'en-1.txt',
 'en-2.txt',
 'en-3.txt',
 'en-4.txt',
 'en-5.txt',
 'fr-10.txt',
 'fr-6.txt',
 'fr-7.txt',
 'fr-8.txt',
 'fr-9.txt',
 'id-11.txt',
 'id-12.txt',
 'id-13.txt',
 'id-14.txt',
 'id-15.txt',
 'tl-16.txt',
 'tl-17.txt',
 'tl-18.txt',
 'tl-19.txt',
 'tl-20.txt']

* 우리 관심은 확장자가 txt로 된 파일

In [22]:
train_files = []

for e in os.listdir(train_path):
    if 'txt' in e:
        train_files.append(e)
        
print(train_files)

['en-1.txt', 'en-2.txt', 'en-3.txt', 'en-4.txt', 'en-5.txt', 'fr-10.txt', 'fr-6.txt', 'fr-7.txt', 'fr-8.txt', 'fr-9.txt', 'id-11.txt', 'id-12.txt', 'id-13.txt', 'id-14.txt', 'id-15.txt', 'tl-16.txt', 'tl-17.txt', 'tl-18.txt', 'tl-19.txt', 'tl-20.txt']


* 테스트 데이터도 학습 데이터와 동일한 방법으로 파일 리스트를 만듦

In [23]:
test_files = []

for e in os.listdir(test_path):
    if 'txt' in e:
        test_files.append(e)
        
print(test_files)

['en-1.txt', 'en-2.txt', 'fr-3.txt', 'fr-4.txt', 'id-5.txt', 'id-6.txt', 'tl-7.txt', 'tl-8.txt']


## 파일 읽기
* 앞서 만든 파일 리스트로부터 파일을 읽어옴

In [36]:
print(train_path, train_files[0])

../data/lang/train en-1.txt


* 연습으로 하나만 읽어오기
* os.path.join(path, \*path)
  * 인자로 넣은 문자들로 이루어진 디렉토리와 파일 경로를 만들어 줌
  * OS에 따라서 경로에 슬래시가 쓰이거나 백슬래시를 사용
  * 현재 프로그램이 돌아가는 OS에 맞추어 알맞은 경로를 만들어줌
* open() 에서 옵션을 그냥 'r'로 주었을 때, 인코딩 에러 발생
  * 해결법: open() 함수에서 encoding 인자에 utf8 줌 -> open(..., encoding='utf8')

In [48]:
with open(os.path.join(train_path, train_files[0]),'r', encoding='utf8') as f:
    lines = f.readlines()

In [51]:
# 읽어온 파일 10줄만 출력해보기
lines[:10]

['\n',
 '\n',
 '\n',
 '\n',
 'The main Henry Ford Museum building houses some of the classrooms for the Henry Ford Academy\n',
 '\n',
 '\n',
 'Henry Ford Academy is the first charter school in the United States to be developed jointly by a global corporation, public education, and a major nonprofit cultural institution. The school is sponsored by the Ford Motor Company, Wayne County Regional Educational Service Agency and The Henry Ford Museum and admits high school students. It is located in Dearborn, Michigan on the campus of the Henry Ford museum. Enrollment is taken from a lottery in the area and totaled 467 in 2010.[1]\n',
 'Freshman meet inside the main museum building in glass walled classrooms, while older students use a converted carousel building and Pullman cars on a siding of the Greenfield Village railroad. Classes are expected to include use of the museum artifacts, a tradition of the original Village Schools. When the Museum was established in 1929, it included a school 

## 오늘의 머신러닝 과정
* 언어 판별에 앞서
  * 동일한 종류의 알파벳(a-z)을 사용하더라도, 언어마다 문자의 출현 빈도가 다름
  * 출현 빈도에 따라 어느 국가 언어로 쓰인 글인지 판별이 가능
* 머신러닝 과정
  * 데이터 수집
    * 이미 되어 있음
  * 데이터 전처리 & 특징 추출
    * 파일 읽어오는 함수 만들기
    * 읽어온 파일 내용을 알파벳만으로 전처리 (data cleansing)
    * 정말 언어마다 알파벳 출현 빈도가 다른지, 알파벳 출현 빈도 히스토그램 그려보기(matplotlib 익힘)
    * 학습할 모델에 적절한 형태로 입력(출현 빈도 벡터)과 출력(국가 원핫 인코딩)을 분리
  * 모델학습
    * 사이킷런에서 제공하는 분류 모델 중에 적용
  * 모델평가
    * 학습한 모델에 테스트 데이터를 넣어 모델 평가

### 파일 읽어오는 함수 만들기
* 함수의 입력과 출력이 무엇이 되어야 하나 먼저 생각
* 파일 종류(학습/테스트)와 파일리스트 인덱스를 주면 글과 국가명을 반환하도록 만들기
* 즉,
  * 입력
    * 학습 / 테스트 디렉토리 경로
    * 각각의 파일 리스트 인덱스
  * 출력
    * 글
    * 국가

In [54]:
def get_file(filepath, filename):
    fullpath = os.path.join(filepath, filename)
    with open(fullpath, 'r', encoding='utf8') as f:
        lines = f.readlines()
    
    temp = str()
    for line in lines:
        temp += line
        
    return temp, filename[:2]

* 만든 함수를 확인 해보기

In [61]:
words, country = get_file(train_path, train_files[0])

In [62]:
words[:100]

'\n\n\n\nThe main Henry Ford Museum building houses some of the classrooms for the Henry Ford Academy\n\n\nH'

In [63]:
country

'en'

## 읽어온 파일 내용을 알파벳만으로 전처리 (data cleansing)
* 앞서 읽어온 데이터에서 알파벳 외의 줄넘김 문자나 숫자 등이 섞여 있음
* 정규표현식을 사용하여 읽어온 내용을 알파벳만으로 전처리
* 정규표현식이란?

In [53]:
# 정규표현식 모듈
import re

In [44]:
re.search('global',temp)

<re.Match object; span=(196, 202), match='global'>

In [36]:
sorted(re.findall('[a-zA-Z]',temp))

['A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D',
 'D'